In [26]:
import gc
import json
import time
import difflib
import requests
import numpy as np
import regex as re
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from collections import Counter
from difflib import SequenceMatcher
from pyphonetics import Metaphone, RefinedSoundex, Soundex, FuzzySoundex

In [27]:
def similar_voter_guardian(x):
    x1 = x.split(' ')
    x1 = [i for i in x1 if i in df]
    x1 = [' '.join(x1) if x1 else x]
    return ' '.join(x1)

In [3]:
fuz = FuzzySoundex()
meta = Metaphone()
ref = RefinedSoundex()
soundex = Soundex()

In [4]:
train = pd.read_csv('/home/shameer/Desktop/Booths/English/089_CSV/008.csv')
train['Voter_name'] = train['Voter_name'].apply(lambda x: x.lower())
train['Guardian_name'] = train['Guardian_name'].apply(lambda x: x.lower())
df = list(train['Voter_name'])
df = [str(i) for i in df]
df = [i.split(' ') for i in df]
df = [j.lower() for i in df for j in i]
d = [i for i in df if i.endswith('amma') | i.endswith('ayya') | i.endswith('iah') | i.endswith('kshmi') | i.endswith('xmi') | i.endswith('reddi') | i.endswith('anna')]
df = [i for i in df if i not in d]
train['new_col'] = train['Voter_name'].apply(lambda x: similar_voter_guardian(x))
sur_names_corpus = list(set(list(pd.read_csv('all.csv')['0'])))
sur_names_corpus = re.compile('\w+').findall(' '.join(sur_names_corpus))
sur_names_corpus = [i for i in sur_names_corpus if len(i)>1]

In [5]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [6]:
 class TrieNode:
     
    
    # Trie node class
    def __init__(self, count, main_key, list1):
        self.children = [None]*26
        self.main_key = main_key
        self.list_strings = list1
        # isEndOfWord is True if node represent the end of the word
        self.isEndOfWord = False
 
 class Trie:
     
    # Trie data structure class
    def __init__(self):
        self.root = self.getNode()
 
    def getNode(self):
     
        # Returns new trie node (initialized to NULLs)
        return TrieNode(0, '', [])
 
    def _charToIndex(self,ch):
         
         
        return ord(ch)-ord('a')
 
 
    def insert(self,key):
         
        pCrawl = self.root
        length = len(key)
        for level in range(length):
            index = self._charToIndex(key[level])
            if index>=0 and index<=26:
                # if current character is not present
                if not pCrawl.children[index]:
                    pCrawl.children[index] = self.getNode()
                    pCrawl.children[index].main_key = key[:level+1]
                if key not in pCrawl.children[index].list_strings:
                        pCrawl.children[index].list_strings.append(key)
                pCrawl = pCrawl.children[index]
 
        # mark last node as leaf
        if key not in pCrawl.list_strings:
                pCrawl.list_strings.append(key)     
        pCrawl.main_key = key
        pCrawl.isEndOfWord = True
 
    def search(self, key):
         
        pCrawl = self.root
        length = len(key)
        ans = []
        for level in range(length):
            index = self._charToIndex(key[level])
            if pCrawl.children[index]:
                ans = pCrawl.children[index].list_strings
                if pCrawl.children[index].main_key == key:
                    return pCrawl.children[index].list_strings
            
            if not pCrawl.children[index]:
                return ans
            pCrawl = pCrawl.children[index]
            
        if pCrawl != None and pCrawl.isEndOfWord:
            return pCrawl.list_strings
        
 

 t = Trie()
 
 # Construct trie
for key in sur_names_corpus:
     t.insert(key)
 

In [7]:
def add(row):
    if ' ' in row['new_col']:
        x = row['new_col'].split(' ')
        x1 = row['Guardian_name'].split(' ')
        for i in x:
            if i in x1:
                return i
        else:
            return None
    else:
        return row['new_col']

In [8]:
train['surs'] = train.apply(add, axis=1)
train['surs'] = train['surs'].replace('', 'None')
train['surs'] = train['surs'].astype(str)

In [9]:
def awl_in(row):
    if len(re.compile('\w+').findall(row['new_col']))>1:
        surs = []
        ratio = 0.85
        ans = None
        x = row['new_col'].split(' ')
        x = re.compile('\w+').findall(' '.join(x))
        dict1 = {}
        for i in x:
            temp = t.search(i[0])
            temp = process.extractBests(i, temp)
            temp = [i[0] for i in temp]
            t1 = difflib.get_close_matches(i, sur_names_corpus)
            for k in t1:
                temp.append(k)
            for j in temp:
                if (meta.sounds_like(i, j) & ref.sounds_like(i, j)) | (fuz.sounds_like(i, j) & ref.sounds_like(i, j)) | (fuz.sounds_like(i, j) & meta.sounds_like(i, j)):
                    surs.append(j)
            for j in surs:
                if similar(i, j) >= ratio:
                    ratio = similar(i, j)
                    if ratio in dict1.keys():
                        dict1[ratio].append(i)
                        dict1[ratio] = list(set(dict1[ratio]))
                    else:
                        dict1[ratio] = [i]  
        if len(dict1)>0:
            max_key = max(dict1, key=lambda k: dict1[k])
            ans = ' '.join(dict1[max_key])
        else:
            ans = None
        return ans
    else:
        return row['new_col']

In [10]:
train['surs'][train['surs']=='None'] = train[train['surs']=='None'].apply(awl_in, axis=1)
train['surs'] = train['surs'].astype(str)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
new_l = list(set(list(train['surs'])))
new_l.remove('None')
for i in new_l:
    t.insert(i)
l = train[train['surs']=='None'].apply(awl_in, axis=1)
for i in list(l.index):
    train['surs'].loc[i] = l.loc[i]
train['surs'] = train['surs'].astype(str)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
train[train['surs']=='None'].shape

(22, 9)

In [14]:
url = (
    "http://rc50-api.nameapi.org/rest/v5.0/parser/personnameparser?"
    "apiKey=f9167e1d1792481e6ca256c37aaf91b0-user1"
)

In [15]:
def nameapi(x):
    payload = {
        "inputPerson": {
            "type": "NaturalInputPerson",
            "personName": {
                "nameFields": [
                    {
                        "string": x,
                        "fieldType": "FULLNAME"}
                ]
            },
            "gender": "UNKNOWN"
        }
    }
    ans = None
    # Proceed, only if no error:
    try:
        time.sleep(1)
        resp = requests.post(url, json=payload)
        resp.raise_for_status()
        # Decode JSON response into a Python dict:
        resp_dict = resp.json()
        ans = resp_dict
    except requests.exceptions.HTTPError as e:
        print("Bad HTTP status code:", e)
    except requests.exceptions.RequestException as e:
        print("Network error:", e)
    try:
        if ans:
            ans = ans['matches'][0]['parsedPerson']['addressingSurname']
    except:
        pass
    return ans

In [16]:
train2 = train[train['surs']=='None']
train2['surs_conf'] = train2['new_col'].apply(lambda x: nameapi(x))
for i in list(train2.index):
    train['surs'].loc[i] = train2['surs_conf'].loc[i]
train['surs'] = train['surs'].astype(str)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
train['surs'] = train['surs'].apply(lambda x: x.lower())

In [20]:
train.to_csv('/home/shameer/Desktop/Booths/English/089_Surs/008.csv', index=False)